<a href="https://colab.research.google.com/github/MHKhan18/colab-notebooks/blob/main/CS_541_B_NYC_TAXI_mar13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 541-B Assignment 2 -- A * search and Uniform Cost Search on NYC Taxi dataset

#### Name: Mohammad Khan
#### Stevens ID: mkhan13@stevens.edu

## NYC Taxi Data
---
The task is to perform Uniform Cost Search and A∗ search. The dataset contains a list of the taxi trips made in NYC in the January 2015. 

Part 1. [20 pts] Represent the data as a graph. For the sake of simplicity, you can assume that edges are only between locations which had a valid taxi trip between them. 

Part 2. [20 pts] Implement the succAndCost fucntion that returns all the connected nodes from the passed node.
And implement the heuristic fucntion using geopy

#### Note: You can install geopy using the command
     pip install geopy

Part 3. [60 pts] 


*   Implement the Uniform Cost search where you can use the trip distances as the edge costs. The program should input two node ids from the user and output the path as well as the cost between them. 
*   Implement A∗ search using a heuristic. One idea of a heuristic value is to use straight line distance between 2 points. This can be computed using the geopy package. The program should input two node ids from the user and output the path as well as the cost between them. 

In [22]:
# Required libraries
import pandas as pd
from tqdm import tqdm
from geopy import distance
import heapq, collections, re, sys, time, os, random

In [2]:
"""
    Download the datset from the below link
"""
!wget 'https://raw.githubusercontent.com/GABudhrani/Assignment_2_NYC_TAxi/main/NYC_dataset.csv'

--2023-03-28 17:27:22--  https://raw.githubusercontent.com/GABudhrani/Assignment_2_NYC_TAxi/main/NYC_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8501184 (8.1M) [text/plain]
Saving to: ‘NYC_dataset.csv’

NYC_dataset.csv     100%[===================>]   8.11M  --.-KB/s    in 0.04s   

2023-03-28 17:27:22 (213 MB/s) - ‘NYC_dataset.csv’ saved [8501184/8501184]



In [3]:
data_NYC = pd.read_csv("NYC_dataset.csv")
data_NYC.head(1)

,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_distance,nodeid1,nodeid2
0,-73.9641,40.7614,-73.978,40.7831,2.4,48293,28440


In [4]:
def plot_nodes_edges(node1, node2, cost, graph):
    """ Input:
              Node 1, Node 2, Cost(	trip_distance) 
              
              example :- 48293, 28440, 2.40
        Output: 
              return graph (Add Key as Node and value as its relevent node and cost)

              Example :- 
              {48293: {2.4: 28440, 1.7: 30199, 0.82: 38931, 3.1: 57087, 1.0: 39197},
               28440: {2.4: 48293, 1.28: 7474, 2.08: 74465, 1.63: 17351}}
              """

    node_cost = {}
    ##############################################START HERE##############################################
    # Part 1 (20 pt)
    if node1 in graph: 
      graph[node1][cost] = node2
    else: 
      graph[node1] = {cost : node2}
    if node2 in graph:
      graph[node2][cost] = node1
    else:
      graph[node2] = {cost: node1}
    ##############################################END HERE################################################
    return graph
    

def create_nodes_edges(dataset):
    graph = {}
    for i in tqdm(range(dataset.shape[0])):
      graph = plot_nodes_edges(dataset.nodeid1[i], dataset.nodeid2[i], dataset.trip_distance[i], graph)
      graph = plot_nodes_edges(dataset.nodeid2[i], dataset.nodeid1[i], dataset.trip_distance[i], graph)
    return graph 

    
def extract_longitude_latitude(dataset):
    """ Input : 
              (nodeid1, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its pickup_longitude & pickup_latitude
              (nodeid2, pickup_latitude, pickup_longitude), Here we want to connect each node 
              with its dropoff_longitude & dropoff_latitude
        Output : 
              {node : [latitude, longitude]} """
    node_lat_long = {}
    for i in tqdm(range(dataset.shape[0])):
        node_lat_long[dataset.nodeid1[i]] = [dataset.pickup_latitude[i],dataset.pickup_longitude[i]]
        node_lat_long[dataset.nodeid2[i]] = [dataset.dropoff_latitude[i],dataset.dropoff_longitude[i]]
    return  node_lat_long

In [5]:
graph = create_nodes_edges(data_NYC)

100%|██████████| 168152/168152 [00:08<00:00, 19481.22it/s]


In [6]:
# calling create_nodes_edges fucntion
node_lat_long = extract_longitude_latitude(data_NYC)

100%|██████████| 168152/168152 [00:08<00:00, 18937.80it/s]


In [7]:
# Data structure for supporting uniform cost search.
class PriorityQueue:
    def  __init__(self):
        self.DONE = -100000
        self.heap = []
        self.priorities = {}  # Map from state to priority

    # Insert |state| into the heap with priority |newPriority| if
    # |state| isn't in the heap or |newPriority| is smaller than the existing
    # priority.
    # Return whether the priority queue was updated.
    def update(self, node, newCost, newHistory):
        oldCost = self.priorities.get(node, None)
        if oldCost == None or newCost < oldCost:
            self.priorities[node] = newCost
            heapq.heappush(self.heap, (newCost, node, newHistory))
            return True
        return False

    # Returns (state with minimum priority, priority)
    # or (None, None) if the priority queue is empty.
    def removeMin(self):
        while len(self.heap) > 0:
            priority, state, history = heapq.heappop(self.heap)
            if self.priorities[state] == self.DONE: continue  # Outdated priority, skip
            self.priorities[state] = self.DONE
            return (state, priority, history)
        return (None, None, None) # Nothing left...


In [8]:
class PathFinding(object):
    def __init__(self, graph, node_lat_long, start_position, end_position):
        self.graph = graph
        self.node_lat_long = node_lat_long
        self.start_node = start_position
        self.end_node = end_position
        
    def startState(self):
        return self.start_node
    
    def succAndCost(self, current_node):
        """ 
            In this Funciton we have to return all the nodes that are connected to the current_node. 
            
            Note that your code needs to work for any input, not just for the one example below. 

            Input:
                  curent_node
            output: 
            children: all the nodes that are connected to the current node and it's cost.
            Example: children = [(node1,cost1), (node2, cost2)]
        """
        # Generate children
        children = []
        ##############################################START HERE##############################################
        # Part 2.1 (10 pt)
        if current_node in self.graph:
          for cost, node in self.graph[current_node].items():
            children.append((node, cost))
        ##############################################END HERE################################################        
        return children
    
    def isEnd(self, current_node):
        return self.end_node == current_node
    
    def heuristic(self, node):
        """ 
          In this function you need to return the heuristic value of node. 
          Input: 
              Input will be the node
          Output: 
              Output will the distance between the node and the end node, please use distance.geodesic to compute the actual distance between two nodes
        """
        ##############################################START HERE##############################################
          # Part 2.2 (10 pt)
        result = distance.geodesic(node_lat_long[node], node_lat_long[self.end_node]).miles

        ##############################################END HERE################################################
        return result

In [9]:
def printSolution(solution):
    totalCost, history = solution
    print_history = 'Starting Node' 
    for node in history:
        print_history += ' -> ' + str(node)
    print(print_history)
    print('totalCost: {}'.format(totalCost))

In [25]:
def astar(problem):
    frontier = PriorityQueue()
    explored = set([])
    """ 
        In this function you have to write an algorithm to calculate the path with the minimum cost using A* Search 
        by calculating it's heuristic cost. Replace the code between the #### with the correct implementation. 
        Input:
              Input will be an object of the class PathFinding
        Output:
              return cost, history
              (history: Path from start node to the end node
              cost: Cost to travel from start node to end node)

                              OR

              return  -1, []
              (if no path is found between Start node and End node)
    """

    ##############################################START HERE##############################################
    # Part 3 (30 pt)
    start_node = problem.startState()
    frontier.update(start_node, problem.heuristic(start_node), [])

    while True:
      path_exists = False
      current_node, past_cost, history = frontier.removeMin()
      if current_node is None:
        break
      explored.add(current_node)
      if problem.isEnd(current_node):
        result = (past_cost, history)
        path_exists = True
        break 
      current_node_heuristics = problem.heuristic(current_node)
      for child_node, cur_node_cost in problem.succAndCost(current_node):
        if child_node in explored:
          continue
        child_heuristic = problem.heuristic(child_node)
        new_distance = past_cost + cur_node_cost + child_heuristic - current_node_heuristics
        frontier.update(child_node, new_distance, history + [child_node]) 

    return result if path_exists else (-1,[])
    ##############################################END HERE################################################
    

In [26]:
def UCS(problem):
    """ 
        In this function you have to write an algorithm to calculate the path with the minimum cost using Uniform Cost Search and UCS 
        by calculating it's heuristic cost, cost respectively. Replace the code between the #### with the correct implementation. 
        Input:
              Input will be an object of the class PathFinding
        Output:
              return cost, history
              (history: Path from start node to the end node
              cost: Cost to travel from start node to end node)

                              OR

              return  -1, []
              (if no path is found between Start node and End node)
    """
    frontier = PriorityQueue()
    explored = set([])
    ##############################################START HERE##############################################
    # Part 3 (30 pt)
    path_exists = False 
    frontier.update(problem.startState(), 0, [])
    while True:
      state, past_cost, history = frontier.removeMin()
      if state is None:
        break
      explored.add(state)
      if problem.isEnd(state):
        result = (past_cost, history)
        path_exists = True
        break
      for new_state, cost in problem.succAndCost(state):
        if new_state in explored:
          continue
        frontier.update(new_state, past_cost + cost, history + [new_state])

    return result if path_exists else (-1,[])
    ##############################################END HERE################################################
   

In [27]:
"""
    Below are some of the test cases that you can try
    Start Node = 129891 | End Node = 7381
    Start Node = 51080 | End Node 79375

"""
start_node = 129891
end_node = 10666
problem = PathFinding(graph, node_lat_long, start_node, end_node) 

In [28]:
# Astart Algorithm
# Calling the function
astart_path_cost = astar(problem)
printSolution(astart_path_cost)

Starting Node -> 134204 -> 102266 -> 47620 -> 10470 -> 140016 -> 121478 -> 87417 -> 96990 -> 93345 -> 7651 -> 46547 -> 10666
totalCost: 15.679999999999998


In [29]:
# UCS Algorithm
# Calling the function
ucs_path_cost = UCS(problem)
printSolution(ucs_path_cost)

Starting Node -> 134204 -> 102266 -> 47620 -> 10470 -> 140016 -> 121478 -> 87417 -> 96990 -> 93345 -> 7651 -> 46547 -> 10666
totalCost: 15.680000000000001


In [16]:
start_node2 = 7381
end_node2 = 10666
problem2 = PathFinding(graph, node_lat_long, start_node2, end_node2) 

In [30]:
ucs_path_cost2 = UCS(problem2)
printSolution(ucs_path_cost2)

Starting Node
totalCost: -1


In [31]:
astart_path_cost2 = astar(problem2)
printSolution(astart_path_cost2)

Starting Node
totalCost: -1


In [32]:
start_node3 = 129891
end_node3 = 7381
problem3 = PathFinding(graph, node_lat_long, start_node3, end_node3) 

In [33]:
ucs_path_cost3 = UCS(problem3)
printSolution(ucs_path_cost3)

Starting Node
totalCost: -1


In [34]:
astart_path_cost3 = astar(problem3)
printSolution(astart_path_cost3)

Starting Node
totalCost: -1


In [35]:
import random

size = len(data_NYC.index)
TEST_SIZE = 20

for i in range(TEST_SIZE):
  from_node_index = random.randint(0, size-1)
  to_node_index = random.randint(0, size-1)
  
  start_node = data_NYC.loc[from_node_index, "nodeid1"]
  end_node = data_NYC.loc[to_node_index, "nodeid2"]
  problem = PathFinding(graph, node_lat_long, start_node, end_node) 

  print(f"###########################Running Test # {i+1}###############################")

  ucs_path_cost = UCS(problem)
  if ucs_path_cost != False:
      print(f"UCS path from {start_node} to {end_node}:")
      printSolution(ucs_path_cost)
  else:
    print(f"No path found in UCS from start node:{start_node} to end node:{end_node}")
  


  astart_path_cost = astar(problem)
  if astart_path_cost != False:
      print(f"A star path from {start_node} to {end_node}:")
      printSolution(astart_path_cost)
  else:
    print(f"No path found in A star from start node:{start_node} to end node:{end_node}")

###########################Running Test # 1###############################
UCS path from 36609 to 124777:
Starting Node -> 58545 -> 17242 -> 50971 -> 91022 -> 75827 -> 107192 -> 1243 -> 67180 -> 136474 -> 49716 -> 106239 -> 49995 -> 40735 -> 143675 -> 124777
totalCost: 15.030000000000001
A star path from 36609 to 124777:
Starting Node -> 58545 -> 17242 -> 50971 -> 91022 -> 75827 -> 107192 -> 1243 -> 67180 -> 136474 -> 49716 -> 106239 -> 49995 -> 40735 -> 143675 -> 124777
totalCost: 15.030000000000001
###########################Running Test # 2###############################
UCS path from 145081 to 13359:
Starting Node -> 134464 -> 75144 -> 51651 -> 134118 -> 19655 -> 58226 -> 84898 -> 4656 -> 137297 -> 8077 -> 134667 -> 44030 -> 88472 -> 13359
totalCost: 93.55
A star path from 145081 to 13359:
Starting Node -> 134464 -> 75144 -> 51651 -> 134118 -> 19655 -> 58226 -> 84898 -> 4656 -> 137297 -> 8077 -> 134667 -> 44030 -> 88472 -> 13359
totalCost: 93.54999999999998
########################